In [1]:
import pickle
import numpy as np
import networkx as nx
from networkx.algorithms import bipartite


def random_bipartite(n_left, n_right, density, seed, mean=None, std=None):
    # Create bipartite random graph
    G = bipartite.random_graph(n_left, n_right, density, seed=seed)

    if (mean is None) or (std is None):
        nx.set_edge_attributes(G, 1, "weight")
    else:
    
        rng = np.random.default_rng(seed)
        n_edges = G.number_of_edges()
        
        # Draw and re-draw until all are nonzero
        int_weights = np.zeros(n_edges, dtype=int)
        mask = np.ones(n_edges, dtype=bool)
        while np.any(mask):
            new_vals = np.round(rng.normal(loc=mean, scale=std, size=np.sum(mask))).astype(int)
            mask_indices = np.where(mask)[0]
            for i, val in zip(mask_indices, new_vals):
                if val != 0:
                    int_weights[i] = val
                    mask[i] = False  # filled successfully
    
        # Assign weights
        for edge, w in zip(G.edges(), int_weights):
            G.edges[edge]["weight"] = w

    return G

In [5]:
n_left = n_right = 128
mean = 50
std = 25

for p in np.arange(0.1, 1.1, 0.1):
    for i in range(1, 11):
        seed = int((p * 10) + i + 40)
        graph = random_bipartite(n_left, n_right, p, seed, mean, std)
        filepath = f"graphs/WG{n_left}_{int(p*10)}_{i}.gpickle"
        with open(filepath, 'wb') as f:
            pickle.dump(graph, f, pickle.HIGHEST_PROTOCOL)